<a href="https://www.kaggle.com/code/wasjaip/lalab-pii-data-7st?scriptVersionId=176385376" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# The Learning Agency Lab - PII Data Detection

In [ ]:
import re
import os
import time

import numpy as np 
import pandas as pd 

import json
import argparse
from itertools import chain
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification
from datasets import Dataset
from colorama import Fore, Style
from torch.utils.data import DataLoader


import gc
import torch
import numpy as np

from scipy.special import softmax

## Настроечные параметры

In [ ]:
# Ставим is_local в True, если локально работаем, если сабмитим - ставим в False
is_local = False
#is_local = True

# Ставим в is_make_predition True если нужно сделать предсказания на инференсе и записать их на диск
# Иначе пользуеся записанными ранее на диск предсказаниями
is_make_predition = False
#is_make_predition = True

# Нужно печатать детали или только итог
#is_print_details = False
is_print_details = True

train_Length = 10000 # Число текстов, которые возбмем для тестированя из тренировочных данных
#train_Length = 100 # Число текстов, которые возбмем для тестированя из тренировочных данных

# Число документов которые передаются в одно предсказание
# и сохраняются на диск
max_batch_size = 500
#max_batch_size = 20

# Максимальная площать токенов на число документов при предсказании в батче
# max_pred_sqaure = 40000*2000*3000
# max_pred_sqaure = 40000*2000*3000*10000

# Лимит времени на выполенния предсказаний моделями по всем документам
#all_docs_time_limit = (8*60*60+60*30)
all_docs_time_limit = (5*60*60)
#all_docs_time_limit = (40*2)

# Лимит времени на выполенния предсказаний моделями по коротким документам
short_docs_time_limit = (8*60*60+60*30)
#short_docs_time_limit = (40*2)

# Лимит времени на выполенния предсказаний моделями по документам c pii
#pii_docs_time_limit = (8*60*60+60*30)
#pii_docs_time_limit = (40*3)

# Лимит времени на выполенния предсказаний моделями по документам c сомнениями
#doubts_docs_time_limit = (8*60*60+60*30)
#doubts_docs_time_limit = (40*1)

# range_share Доля документов которые обработаем.
# Обрабатываем сначала с более коротких документов
range_share = 2/3
#range_share = 9/10

# Порог вероятности метки O для финального предсказания
#all_docs_threshold = 0.9735
final_docs_threshold = 0.93

# Порог вероятности метки O для поиска документов с pii
# Это предварительный порог. Чтобы только найти документы в которых потом еще искать
# Персональную информацию
#all_docs_threshold = 0.9735
find_pii_docs_threshold = 0.93

# Пороги раздельно по классам
thresholds_unique = [0.15, #'B-EMAIL',
                     0.15, #'B-ID_NUM',
                     0.10, #'B-NAME_STUDENT',
                     0.15, #'B-PHONE_NUM',
                     0.15, #'B-STREET_ADDRESS',
                     0.15, #'B-URL_PERSONAL',
                     0.15, #'B-USERNAME',
                     0.15, #'I-ID_NUM',
                     0.10, #'I-NAME_STUDENT',
                     0.15, #'I-PHONE_NUM',
                     0.15, #'I-STREET_ADDRESS',
                     0.15, #'I-URL_PERSONAL',
                     0]    #'O'
# Пороги раздельно по классам

In [ ]:
# Время старта работы ноутбука
notebook_starttime = time.time()

In [ ]:
# Возвращает сколько уже работает ноутбук
def p_time():
    #
    run_time = round(time.time() - notebook_starttime)
    return str(run_time).zfill(5)+' sec:'

In [ ]:
print(p_time(), 'start notebook')

## Предсказание меток токенов

### Предсказанием меток токенов трансформерами.

In [ ]:
# Максимальная длина текста для вывода
# INFERENCE_MAX_LENGTH = 4500

def tokenize(example, tokenizer):
    
    # Максимальная длина текста для вывода
    INFERENCE_MAX_LENGTH = 3500
    text = [] # Инициализация списка для хранения текста
    token_map = [] # Инициализация списка для сопоставления индексов токенов
    
    idx = 0 # Начальный индекс для отслеживания токенов
    
    # Перебор токенов и связанных с ними пробелов после токенов
    for t, ws in zip(example["tokens"], example["trailing_whitespace"]):
        
        text.append(t) # Добавление каждого токена в список 'text'
        
        # Расширение 'token_map' текущим индексом, повторенным столько раз, сколько символов в токене
        token_map.extend([idx] * len(t))
        
        # Добавление пробела для следующего пробела и отметка его '-1' в 'token_map'
        if ws:
            text.append(" ")
            token_map.append(-1)
            
        idx += 1 # Увеличение 'idx' для следующего токена
        
    # Токенизация объединенного текста 'text' и возвращение смещений вместе с 'token_map'
    tokenized = tokenizer("".join(text), return_offsets_mapping=True, truncation=True, max_length=INFERENCE_MAX_LENGTH)
    
    # Возвращение словаря, содержащего токенизированные данные и 'token_map'
    return {
        **tokenized,
        "token_map": token_map,
    }


In [ ]:
# Загрузка данных из файла JSON

# Локально работаем с тренировочной выборкой
# При сабмите загружаекм тестовую выборку
data_path = ('/kaggle/input/pii-detection-removal-from-educational-data/train.json' if is_local
             else '/kaggle/input/pii-detection-removal-from-educational-data/test.json')
test_data = json.load(open(data_path))

# Сортировка test_data по числу токенов в списке tokens каждого элемента
# Предположительно это до должно ускорить инференс за счет более ровного размера тензоров в батче
test_data = sorted(test_data, key=lambda x: len(x['tokens']))

if is_local:
    # Возьмем для проверки не все тренировочные данные. А только часть. train_Length
    test_data = test_data[0:train_Length]

if is_local:
    # Создадим отдельный датафрейм df_result дли исхходных размеченных данных.
    # Чтобы потом сравнить какие персональные данные были найдены, а какие нет
    # А может что лишнего был онайдено
    df_result = pd.DataFrame({
        "document": [x["document"] for x in test_data], # Получение номеров документов
        "tokens": [x["tokens"] for x in test_data], # Получение токенов
        "true_labels": [x["labels"] for x in test_data], # Получение меток токенов
        "pred_labels": [['O'] * len(x["labels"]) for x in test_data], # Пока все предсказанные метки заполняем 'О'. Так как еще не нашли персональную информацию
        "pred_ind": [[-1] * len(x["labels"]) for x in test_data], # Индекс соответствующего токена в предсказании
    })
    

'''
model_paths = {
    '/kaggle/input/37vp4pjt': 10,
    '/kaggle/input/pii-deberta-models/cuerpo-de-piiranha': 2,
    '/kaggle/input/pii-deberta-models/cola del piinguuino' : 1,
    '/kaggle/input/pii-deberta-models/cabeza-del-piinguuino': 5,
    '/kaggle/input/pii-deberta-models/cabeza-de-piiranha': 3,
    '/kaggle/input/pii-deberta-models/cola-de-piiranha':1,
    '/kaggle/input/pii-models/piidd-org-sakura': 2,
    '/kaggle/input/pii-deberta-models/cabeza-de-piiranha-persuade_v0':1,
}
'''
#'/kaggle/input/pii-v3-hf':1,
model_paths = {
    #'/kaggle/input/pii-v3-hf':1,
    '/kaggle/input/37vp4pjt': 5,
    '/kaggle/input/pii-deberta-models/cabeza-del-piinguuino': 2,
    '/kaggle/input/pii-deberta-models/cola del piinguuino': 2,
    '/kaggle/input/pii-deberta-models/cola-de-piiranha': 2,
    '/kaggle/input/pii-models/piidd-org-sakura': 2,
    '/kaggle/input/pii-deberta-models/cuerpo-de-piiranha': 3,
    '/kaggle/input/pii-deberta-models/cabeza-de-piiranha': 2,
    '/kaggle/input/pii-deberta-models/cabeza-de-piiranha-persuade_v0': 1,
    # Дополнительные модели и пути:
    '/kaggle/input/pii-deberta-models/cabeza-de-piiranha-mixtral-v1': 1,
    '/kaggle/input/pii-deberta-models/cola-de-piiranha-mixtral-v1': 1,
    '/kaggle/input/pii-deberta-models/cuerpo-de-piiranha-mixtral-v1': 1,
    '/kaggle/input/pii-deberta-models/piiranha': 1, 
    '/kaggle/input/pii-deberta-models/piranha': 1,  
    '/kaggle/input/pii-deberta-models/cuerpo del piinguuino': 1,
}



# Выбор первого пути модели из словаря
first_model_path = list(model_paths.keys())[0]

# Инициализация токенизатора из первой модели
tokenizer = AutoTokenizer.from_pretrained(first_model_path)


In [ ]:
# Вычисляем общий вес
total_weight = sum(model_paths.values())

# Список документов
doc_list = [x["document"] for x in test_data]

# Директория для сохранения промежуточных предсказаний всех моделей
intermediate_dir = './model_predictions'
os.makedirs(intermediate_dir, exist_ok=True)  # Создаем директорию, если она не существует

# Директория для сохранения промежуточных предсказаний текущей модели
predict_dir = './predict_dir'
os.makedirs(predict_dir, exist_ok=True)  # Создаем директорию, если она не существует

config = json.load(open(Path(first_model_path) / "config.json"))  # Загружаем конфигурационный файл модели
#! id2label = config["id2label"]  # Извлекаем словарь для преобразования id в метки
id2label = {'0': 'B-EMAIL',
            '1': 'B-ID_NUM',
            '2': 'B-NAME_STUDENT',
            '3': 'B-PHONE_NUM',
            '4': 'B-STREET_ADDRESS',
            '5': 'B-URL_PERSONAL',
            '6': 'B-USERNAME',
            '7': 'I-ID_NUM',
            '8': 'I-NAME_STUDENT',
            '9': 'I-PHONE_NUM',
            '10': 'I-STREET_ADDRESS',
            '11': 'I-URL_PERSONAL',
            '12': 'O'}

weights = list(model_paths.values())
weights

In [ ]:
# Делает датасет для обучения
# docs_to_process - список указывающий обрабатывать документ или нет.
# если стоит 1 - обрабатывать, если 0 - пропускаем документ и не включаем в датасет
def make_dataset(docs_to_process):
    global ds, ds_tokens_len
    # Создание набора данных из загруженных данных
    if 'ds'in globals():
        del ds
    ds = Dataset.from_dict({
        "full_text": [x["full_text"] for idx, x in enumerate(test_data) if docs_to_process[idx] == 1], # Получение текста из каждого элемента
        "document": [x["document"] for idx, x in enumerate(test_data) if docs_to_process[idx] == 1], # Получение номеров документов
        "tokens": [x["tokens"] for idx, x in enumerate(test_data) if docs_to_process[idx] == 1], # Получение токенов
        "trailing_whitespace": [x["trailing_whitespace"] for idx, x in enumerate(test_data) if docs_to_process[idx] == 1], # Получение информации о пробелах
    })
    # Токенизация набора данных с использованием функции 'tokenize' в параллельном режиме
    ds = ds.map(tokenize, fn_kwargs={"tokenizer": tokenizer}, num_proc=2)
    ds_tokens_len = [len(x["tokens"]) for idx, x in enumerate(test_data) if docs_to_process[idx] == 1]

In [ ]:
# Возвращает только ту часть предсказаний, которые указаны в docs_to_process
def get_pred_part(predictions, docs_to_process):
    # Создаем маску, чтобы найти индексы строк, которые нужно оставить
    indices_to_keep = np.where(np.array(docs_to_process) == 1)[0]
    return predictions[indices_to_keep]

In [ ]:
def make_convert_list():
    standart_label2id ={
        'B-EMAIL': 0,
        'B-ID_NUM': 1,
        'B-NAME_STUDENT': 2,
        'B-PHONE_NUM': 3,
        'B-STREET_ADDRESS': 4,
        'B-URL_PERSONAL': 5,
        'B-USERNAME': 6,
        'I-ID_NUM': 7,
        'I-NAME_STUDENT': 8,
        'I-PHONE_NUM': 9,
        'I-STREET_ADDRESS': 10,
        'I-URL_PERSONAL': 11,
        'O': 12
    } 
    # Словарь для преобразования меток
    label_mapping = {
        'O': 'O',
        'B-CITY': 'O',
        'I-CITY': 'O',
        'B-FIRSTNAME': 'B-NAME_STUDENT',
        'I-FIRSTNAME': 'I-NAME_STUDENT',
        'B-USERNAME': 'B-USERNAME',
        'I-USERNAME': 'B-USERNAME',
        'B-JOBTYPE': 'O',
        'B-PREFIX': 'O',
        'I-PREFIX': 'O',
        'B-LASTNAME': 'I-NAME_STUDENT',
        'B-EMAIL': 'B-EMAIL',
        'I-EMAIL': 'B-EMAIL',
        'B-NEARBYGPSCOORDINATE': 'O',
        'I-NEARBYGPSCOORDINATE': 'O',
        'B-ACCOUNTNUMBER': 'O',
        'I-ACCOUNTNUMBER': 'O',
        'B-ACCOUNTNAME': 'O',
        'I-ACCOUNTNAME': 'O',
        'B-MIDDLENAME': 'I-NAME_STUDENT',
        'I-MIDDLENAME': 'I-NAME_STUDENT',
        'B-COUNTY': 'O',
        'I-COUNTY': 'O',
        'B-AGE': 'O',
        'B-CREDITCARDCVV': 'O',
        'B-DOB': 'O',
        'I-DOB': 'O',
        'B-MASKEDNUMBER': 'O',
        'I-MASKEDNUMBER': 'O',
        'B-PASSWORD': 'O',
        'I-PASSWORD': 'O',
        'B-SEX': 'O',
        'B-STATE': 'O',
        'B-COMPANYNAME': 'O',
        'I-COMPANYNAME': 'O',
        'B-PHONEIMEI': 'B-ID_NUM',
        #'B-PHONEIMEI': 'O',
        'I-PHONEIMEI': 'I-ID_NUM',
        #'I-PHONEIMEI': 'O',
        'B-STREET': 'B-STREET_ADDRESS',
        'I-STREET': 'I-STREET_ADDRESS',
        'B-SSN': 'O',
        'I-SSN': 'O',
        'B-IPV4': 'O',
        'I-IPV4': 'O',
        'B-USERAGENT': 'O',
        'I-USERAGENT': 'O',
        'B-MAC': 'O',
        'I-MAC': 'O',
        'B-PIN': 'O',
        'I-PIN': 'O',
        'B-IP': 'O',
        'I-IP': 'O',
        'B-URL': 'B-URL_PERSONAL',
        'I-URL': 'I-URL_PERSONAL',
        'B-CURRENCYSYMBOL': 'O',
        'B-DATE': 'O',
        'I-DATE': 'O',
        'B-TIME': 'O',
        'I-TIME': 'O',
        'B-VEHICLEVRM': 'O',
        'I-VEHICLEVRM': 'O',
        'I-AMOUNT': 'O',
        'B-ETHEREUMADDRESS': 'O',
        'I-ETHEREUMADDRESS': 'O',
        'B-BITCOINADDRESS': 'O',
        'I-BITCOINADDRESS': 'O',
        'B-LITECOINADDRESS': 'O',
        'I-LITECOINADDRESS': 'O',
        'I-JOBTYPE': 'O',
        'B-CREDITCARDNUMBER': 'O',
        'I-CREDITCARDNUMBER': 'O',
        'B-IPV6': 'O',
        'I-IPV6': 'O',
        'I-LASTNAME': 'O',
        'B-PHONENUMBER': 'B-PHONE_NUM',
        'I-PHONENUMBER': 'I-PHONE_NUM',
        'B-CREDITCARDISSUER': 'O',
        'I-CREDITCARDISSUER': 'O',
        'B-SECONDARYADDRESS': 'O',
        'I-SECONDARYADDRESS': 'O',
        'B-ZIPCODE': 'O',
        'I-ZIPCODE': 'O',
        'B-VEHICLEVIN': 'O',
        'I-VEHICLEVIN': 'O',
        'I-AGE': 'O',
        'B-GENDER': 'O',
        'I-GENDER': 'O',
        'B-ORDINALDIRECTION': 'O',
        'B-JOBAREA': 'O',
        'B-HEIGHT': 'O',
        'I-HEIGHT': 'O',
        'B-JOBTITLE': 'O',
        'I-JOBTITLE': 'O',
        'B-BUILDINGNUMBER': 'O',
        'I-BUILDINGNUMBER': 'O',
        'B-AMOUNT': 'O',
        'I-STATE': 'O',
        'I-CURRENCYSYMBOL': 'O',
        'B-IBAN': 'O',
        'I-IBAN': 'O',
        'B-BIC': 'O',
        'I-BIC': 'O',
        'B-EYECOLOR': 'O',
        'B-CURRENCYNAME': 'O',
        'I-CURRENCYNAME': 'O',
        'B-CURRENCY': 'O',
        'I-CURRENCY': 'O',
        'B-CURRENCYCODE': 'O',
        'I-CURRENCYCODE': 'O',
        'I-JOBAREA': 'O',
        'I-EYECOLOR': 'O',
        'I-CREDITCARDCVV': 'O'
    }
    # Создаем пустой список для конвертации
    convert_list = [[] for _ in range(13)]
    # Заполняем список для конвертации меток
    for idx, (in_label, standart_label) in enumerate(label_mapping.items()):
        convert_list[standart_label2id[standart_label]].append(idx)
    return convert_list

In [ ]:
# Конвертирует предсказания из 54 классов в 13 классов
def convert_predition(predictions):
    # Создаем предсказания стандартной размерности.
    pred_standart = np.zeros((*predictions.shape[:-1], 13))
    # Конвертируем предсказания из 54 классов в 13 классов
    convert_list = make_convert_list()
    predictions = softmax(predictions, axis=-1) 
    for idx, cols in enumerate(convert_list):
        if cols:
            # Выбираем срезы по последнему измерению из массива predictions и суммируем их
            pred_standart[:,:, idx] = np.sum(predictions[:,:, cols], axis=-1)
    return pred_standart

In [ ]:
# Выравнивает второе измерение у двух трехмерных numpy массивов
def align_dimension_2(array_1, array_2):
    max_cols = max(array_1.shape[1], array_2.shape[1])
            
    #print('shape', array_1.shape, array_2.shape)
    # Дополняем массивы нулями до максимальных размерностей
    array_1 = np.pad(array_1,
                     ((0, 0),
                      (0, max_cols - array_1.shape[1]),
                      (0, 0)),
                     mode='constant')
    
    array_2 = np.pad(array_2,
                     ((0, 0),
                      (0, max_cols - array_2.shape[1]),
                      (0, 0)),
                     mode='constant')

    return(array_1, array_2)

In [ ]:
# Подготавливаем трейнер и все что необходимо для него
def prepair_trainer(model_path):
    global tokenizer, model, collator, args, trainer
    
    model = AutoModelForTokenClassification.from_pretrained(model_path)  # Загружаем модель для классификации токенов
    collator = DataCollatorForTokenClassification(tokenizer, pad_to_multiple_of=16)  # Создаем объект для подготовки данных к обучению
    
    args = TrainingArguments(  # Устанавливаем аргументы для обучения
        ".",
        per_device_eval_batch_size=1,
        report_to="none",
        disable_tqdm=True,
    )

    trainer = Trainer(  # Создаем объект тренера для управления процессом обучения и оценки
        model=model,
        args=args,
        data_collator=collator,
        tokenizer=tokenizer,
    )

In [ ]:
# Получаем предсказания дробя на батчи
def get_batched_predition(model_path):
    global tokenizer, model, collator, args, trainer
    #print(p_time(), 'start prepair trainer')
    prepair_trainer(model_path)
    #print(p_time(), 'stop prepair trainer')


    # Будем делать предсказания по батчам и сохранять предсказания
    # Индекс в датасете
    ds_idx = 0
    # Индекс файла с предсказаниями
    file_idx = 0
    while ds_idx < len(ds):
        # Динамически ищем размер батча, чтобы не слишком маленький. но и не забить память
        # Размер батча будет тем меньше чем больше токенов
        end_batch_idx = ds_idx
        while ((end_batch_idx < len(ds_tokens_len))
               # Площадь площадь токенов на число документов в предсказании не превышает лимит
               #and ((ds_tokens_len[end_batch_idx]*ds_tokens_len[end_batch_idx]*ds_tokens_len[end_batch_idx]*(end_batch_idx-ds_idx)) < max_pred_sqaure)
               and ((end_batch_idx-ds_idx) < max_batch_size)):
            end_batch_idx += 1

        start_predict = time.time()
        predictions = trainer.predict(
            ds.select(range(ds_idx,end_batch_idx))).predictions  # Получаем предсказания модели на нашем наборе данных
        if is_local:
            print('start index:', ds_idx,
                  'tokens_len:', np.round(np.mean(np.array(
                      ds_tokens_len[ds_idx:end_batch_idx]))).astype(int),
                  'batch_size:', end_batch_idx-ds_idx,
                  'batch_time:', round((time.time() - start_predict), 2))
        if predictions.shape[2] > 13:
            # Если модель предсказывает больше 13 классов трансформируем предсказания в 13 классов.
            predictions = convert_predition(predictions)
        np.savez_compressed(os.path.join(predict_dir, f'preds_{file_idx}.npz'), arr=predictions)
        file_idx +=1
        ds_idx = end_batch_idx
        del predictions  # Удаляем созданные объекты для освобождения памяти
        torch.cuda.empty_cache()  # Очищаем кэш CUDA, чтобы освободить память GPU
        gc.collect()  # Вызываем сборщик мусора для очистки неиспользуемой памяти
        # Ваш код обработки батча
    del trainer, args, collator, model  # Удаляем созданные объекты для освобождения памяти
    torch.cuda.empty_cache()  # Очищаем кэш CUDA, чтобы освободить память GPU
    gc.collect()
    # Теперь загружаем пресказания, сохраненные ранее на диске
    for idx in range(file_idx):
        # print(idx)
        cur_predictions = np.load(os.path.join(predict_dir, f'preds_{idx}.npz'))['arr']
        if idx == 0:
            predictions = cur_predictions
        else:
            # Выравниваем массивы
            # predictions, cur_predictions = align_dimension_2(predictions, cur_predictions)

            max_cols = max(predictions.shape[1], cur_predictions.shape[1])
                    
            #print('shape', array_1.shape, array_2.shape)
            # Дополняем массивы нулями до максимальных размерностей
            predictions = np.pad(predictions,
                                 ((0, 0),
                                  (0, max_cols - predictions.shape[1]),
                                  (0, 0)),
                                 mode='constant')
            
            cur_predictions = np.pad(cur_predictions,
                                     ((0, 0),
                                      (0, max_cols - cur_predictions.shape[1]),
                                      (0, 0)),
                                     mode='constant')
            
            # Объединяем массивы
            predictions = np.concatenate((predictions, cur_predictions))
            del cur_predictions
    return predictions

In [ ]:
def modify_array(arr):
    arr += 1

original_array = np.array([1, 2, 3])
print("Original array:", original_array)

modify_array(original_array)
print("Array after modification:", original_array)

In [ ]:
# Делает предсказание для модели с индексом idx в model_paths
def make_predition(idx, model_path, docs_to_process):
    global weighted_predictions
    #global raw_model_predictions
    if is_make_predition or not is_local:
        # Если указано чделать предсказания на инференсе или если выполняем не локально, а сабми
        # то в любом случае делаем предсказания и записываем их на диск. Иначе пользуемся теми предсказаниями
        # что были записаны на диске ранее
        
        predictions = get_batched_predition(model_path)
    
        #del model, trainer, tokenizer  # Удаляем созданные объекты для освобождения памяти
        #torch.cuda.empty_cache()  # Очищаем кэш CUDA, чтобы освободить память GPU
        #gc.collect()

        #if is_local:
            # Сохраним предсказания модели в список
            #raw_model_predictions.append(predictions.copy())

        # Конвертирует предсказания из 54 классов в 13 классов
        #! predictions = convert_predition(predictions)
        # predictions = convert_predition(predictions)
        
        weighted_predictions = softmax(predictions, axis=-1)  # Применяем softmax к предсказаниям и умножаем на вес модели

        if is_local:
            # Сохраняем взвешенные предсказания на диск
            np.savez_compressed(os.path.join(intermediate_dir, f'weighted_preds_model_{idx}.npz'), arr=weighted_predictions)
        
        # Очищаем память
        del predictions  # Удаляем созданные объекты для освобождения памяти
        torch.cuda.empty_cache()  # Очищаем кэш CUDA, чтобы освободить память GPU
        gc.collect()  # Вызываем сборщик мусора для очистки неиспользуемой памяти
    elif is_local:
        # Загружаем взвешенные предсказания модел
        weighted_predictions = np.load(os.path.join(intermediate_dir, f'weighted_preds_model_{idx}.npz'))['arr']
        weighted_predictions = get_pred_part(weighted_predictions, docs_to_process)

In [ ]:
# Делает суммарное предсказание, на указанных документах,
# с указанным укзанием по времени предсказания
# start_idx - индекс модели в model_paths с которой начинать предсказание
# docs_to_process - список в котором указано какие документы обрабатывать
# time_limit - порог времения выполнения ноубтука, которое нельзя превысить
# sum_predictions_name - наименование файла в который записывать суммарные предсказания
# model_time_mul при вычислениии остатка времени максимальное время модели домножается на это число

def make_sum_predition(start_idx,
                       docs_to_process,
                       time_limit,
                       sum_predictions_name,
                       model_time_mul):
#def make_sum_predition(start_idx, time_limit, sum_predictions_name):
    global weighted_predictions, model_predictions
    #global raw_model_predictions
    
    max_model_time = 0 # Время которое тренировалась модель
    grp = []
    model_weights = 0  # Суммарный вес моделей в группе
    # Список с прекдсказаниями моделей
    if is_local:
        model_predictions = []
        #raw_model_predictions = []
    for idx, (model_path, weight) in enumerate(model_paths.items()):  # Итерируемся по путям и весам моделей
    
        if idx < start_idx:
            continue
        start_model_time = time.time()
        print(p_time(), f'start {model_path}')
        
        make_predition(idx, model_path, docs_to_process)
    
         #Добавляем оттренированную модель к индексу
        grp.append(idx)
    
        if is_local:
            # Сохраним предсказания модели в список
            model_predictions.append(weighted_predictions.copy())
    
        weighted_predictions *= weights[idx]
        model_weights += weights[idx]  # Добавляем вес текущей модели к общему весу моделей в группе
        
        if idx > start_idx:
            #Загружаем накопленные предсказания, если уже не первый цикл
            sum_predictions = np.load(os.path.join(intermediate_dir, f'{sum_predictions_name}.npz'))['arr']
            #sum_predictions, weighted_predictions = align_dimension_2(sum_predictions, weighted_predictions)
            
            max_cols = max(sum_predictions.shape[1], weighted_predictions.shape[1])
            # Дополняем массивы нулями до максимальных размерностей
            sum_predictions = np.pad(sum_predictions,
                                     ((0, 0),
                                      (0, max_cols - sum_predictions.shape[1]),
                                      (0, 0)),
                                     mode='constant')
            
            weighted_predictions = np.pad(weighted_predictions,
                                          ((0, 0),
                                          (0, max_cols - weighted_predictions.shape[1]),
                                          (0, 0)),
                                         mode='constant')
            
        # Накапливаем предсказания, суммируя их
        sum_predictions = (weighted_predictions if idx == start_idx
                                        else sum_predictions + weighted_predictions) 
        # Сохраняем агрегированные предсказания на диск
        np.savez_compressed(os.path.join(intermediate_dir, f'{sum_predictions_name}.npz'), arr=sum_predictions)
        del weighted_predictions, sum_predictions
        
        cur_time = time.time()
        print('model time:', cur_time - start_model_time)
        max_model_time = max(max_model_time, cur_time - start_model_time)
        # Смотрим, достаточно ли времени для тренировки еще одной модели и чтобы остался после этого еще час
        if ((cur_time - notebook_starttime) > (time_limit - (max_model_time*model_time_mul))):
        #if ((cur_time - notebook_starttime) > (40*2 - max_model_time)):
            print('Превышено время для тренеровок моделей')
            print('ноутбук выполнялся:', (cur_time - notebook_starttime))
            print('max_model_time:', max_model_time)
            print('Оттренированы модели:', grp)
            break
    
    #weighted_average_predictions = np.load(os.path.join(intermediate_dir, f'{sum_predictions_name}.npz'))['arr']
    #weighted_average_predictions /= model_weights
    return model_weights, idx, max_model_time

In [ ]:
# Возвращает предсказания с установленным порогом threshold
def get_preds_final(threshold):
    global weighted_average_predictions
    
    preds = weighted_average_predictions.argmax(-1)  # Получаем предсказания, выбирая максимальное значение по последнему измерению

    preds_without_O = weighted_average_predictions[:,:,:12].argmax(-1)  # Получаем предсказания без учета класса 'O', выбирая максимумы до 12-го индекса
    
    O_preds = weighted_average_predictions[:,:,12]  # Извлекаем предсказания для класса 'O' (12-й столбец)
    
    return np.where(O_preds < threshold, preds_without_O, preds), O_preds

### Возвращает предсказания с индивидуальным для каждого класса threshold

In [ ]:
# Возвращает предсказания с индивидуальным для каждого класса threshold,
def get_preds_final_unique():
    global weighted_average_predictions
    #global thresholds_expanded, mask, weighted_average_predictions_copy, preds, preds_without_O, preds_without_O_max
    # Повторяем массив порогов по размерностям массива данных
    #weighted_average_predictions_copy = weighted_average_predictions.copy()
    for idx in range(weighted_average_predictions.shape[0]):
        thresholds_expanded = np.tile(
            np.array(thresholds_unique),
            (weighted_average_predictions.shape[1], 1))
        
        # Создаем маску, где True соответствует элементам массива, которые меньше соответствующих порогов
        mask = weighted_average_predictions[idx] < thresholds_expanded
        
        # Зануляем элементы массива, удовлетворяющие маске
        weighted_average_predictions[idx][mask] = 0
        del mask, thresholds_expanded
    
    preds = weighted_average_predictions.argmax(-1)
    preds_without_O = weighted_average_predictions[:,:,:12].argmax(-1)
    preds_without_O_max = weighted_average_predictions[:,:,:12].max(-1)
    O_preds = weighted_average_predictions[:,:,12]  # Извлекаем предсказания для класса 'O' (12-й столбец)
    
    return np.where(preds_without_O_max > 0, preds_without_O, preds), O_preds

In [ ]:
#weighted_average_predictions[0,0]

In [ ]:
#weighted_average_predictions_copy[0,0]

In [ ]:
#mask[0,0]

In [ ]:
#Возвращает список где единицами указаны документы в которых есть 
# некоторые сомнения есть ли перс информация или нет. 
def get_docs_with_doubts():
    global weighted_average_predictions
    
    name_count = 0
    no_name_count = 0
    O_preds = weighted_average_predictions[:,:,12]
    # В этом списке 1 будет в документе где есть персональная информация,
    # а ноль, если нет персоанльной информации
    docs_with_doubts = [0] * len(doc_list)
    for i, input_ids in enumerate(ds['input_ids']):
        # Смотрим нашлим ли хоть одино не 'O' токен
        if np.any((O_preds[i][0:len(input_ids)] < find_pii_docs_threshold) & (O_preds[i][0:len(input_ids)] > .5)):
            # Если в документе есть PII токены.
            name_count += 1
            docs_with_doubts[i] = 1
        else:
            # Если в документе только токены O найдены
            no_name_count += 1
    
    print('doc without doubts:', no_name_count, '; doc with doubts:', name_count)
    return(docs_with_doubts)

In [ ]:
'''
# Создаем случайный трехмерный массив для примера
array = np.random.randint(0, 100, size=(5, 5, 13))

# Список порогов
#thresholds = [10, 20, 99, 40, 96, 60, 70, 80, 90, 10, 20, 30, 40]
thresholds = [96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 0]

# Повторяем массив порогов по размерностям массива данных
thresholds_expanded = np.tile(np.array(thresholds), (array.shape[0], array.shape[1], 1))

# Создаем маску, где True соответствует элементам массива, которые меньше соответствующих порогов
mask = array < thresholds_expanded

# Зануляем элементы массива, удовлетворяющие маске
weighted_average_predictions = array.copy()
weighted_average_predictions[mask] = 0

preds = weighted_average_predictions.argmax(-1)
preds_without_O = weighted_average_predictions[:,:,:12].argmax(-1)
preds_without_O_max = weighted_average_predictions[:,:,:12].max(-1)
ret = np.where(preds_without_O_max > 0, preds_without_O, preds)
#print(array1)
'''
pass

In [ ]:
#Возвращает список где единицами указаны документы в которых есть персональная информация
def get_docs_with_pii(preds_final):
    name_count = 0
    no_name_count = 0
    # В этом списке 1 будет в документе где есть персональная информация,
    # а ноль, если нет персоанльной информации
    docs_with_pii = [0] * len(doc_list)
    for i, input_ids in enumerate(ds['input_ids']):
        # Смотрим нашлим ли хоть одино не 'O' токен
        if np.any(preds_final[i][0:len(input_ids)] != 12):
            # Если в документе есть PII токены.
            name_count += 1
            docs_with_pii[i] = 1
        else:
            # Если в документе только токены O найдены
            no_name_count += 1
    
    print('doc without names:', no_name_count, '; doc with names:', name_count)
    return(docs_with_pii)

In [ ]:
#Возвращает список где единицами указаны документы которые попали в указанный диапазон
def get_docs_in_range(range_start, range_end):
    docs_in_range = [0] * len(doc_list)
    docs_in_range[range_start:range_end] = [1] * (range_end - range_start)
    
    return(docs_in_range)

In [ ]:
# Собираем 
def make_average_predictions():
    global weighted_average_predictions
    # Добавим к предсказаниям на всем датасете, предсказания дополнительных моделей только на текстах с pii
    weighted_average_predictions = np.load(os.path.join(intermediate_dir, f'all_sum_predictions.npz'))['arr']
    if has_short_doc:
        short_sum_predictions = np.load(os.path.join(intermediate_dir, f'short_sum_predictions.npz'))['arr']
        # Выравниваем вторые размерности моссивов (число токенов в документах
        weighted_average_predictions, short_sum_predictions = align_dimension_2(weighted_average_predictions, short_sum_predictions)
    '''
    if has_pii_doc:
        pii_sum_predictions = np.load(os.path.join(intermediate_dir, f'pii_sum_predictions.npz'))['arr']
        # Выравниваем вторые размерности моссивов (число токенов в документах
        weighted_average_predictions, pii_sum_predictions = align_dimension_2(weighted_average_predictions, pii_sum_predictions)

    if has_doubts_doc:
        doubts_sum_predictions = np.load(os.path.join(intermediate_dir, f'doubts_sum_predictions.npz'))['arr']
        weighted_average_predictions, doubts_sum_predictions = align_dimension_2(weighted_average_predictions, doubts_sum_predictions)
    '''

    #pii_idx = 0
    #doubts_idx = 0
    short_idx = 0
    for weighted_idx in range(weighted_average_predictions.shape[0]):
        cur_model_weights = all_model_weights
        if has_short_doc:
            if docs_with_short[short_idx] == 1:
                #Есди в документе есть персональныя информация
                cur_model_weights += short_model_weights
                weighted_average_predictions[weighted_idx] += short_sum_predictions[short_idx]
                short_idx += 1
        '''
        if has_pii_doc:
            if docs_with_pii[weighted_idx] == 1:
                #Есди в документе есть персональныя информация
                cur_model_weights += pii_model_weights
                weighted_average_predictions[weighted_idx] += pii_sum_predictions[pii_idx]
                pii_idx += 1
        if has_doubts_doc:
            if docs_with_doubts[weighted_idx] == 1:
                #Есди в документе есть токены с сомнениями
                cur_model_weights += doubts_model_weights
                weighted_average_predictions[weighted_idx] += doubts_sum_predictions[doubts_idx]
                doubts_idx += 1
        '''  
        weighted_average_predictions[weighted_idx] /= cur_model_weights

### Главный цикл предсказаний

In [ ]:
%%time

# Отмечаем что в начале обрабатываем все документы (едеиницы в каждом элементе списка всех документов) 
print(p_time(), 'Тренируем модели на всем датасете')
all_docs = [1] * len(doc_list)
make_dataset(all_docs)
all_model_weights, last_idx, max_model_time = make_sum_predition(
    start_idx=0,
    docs_to_process=all_docs,
    time_limit=all_docs_time_limit,
    sum_predictions_name='all_sum_predictions',
    model_time_mul=2
)
print ('all_model_weights:',all_model_weights,'last_idx:',last_idx)
weighted_average_predictions = np.load(os.path.join(intermediate_dir, f'all_sum_predictions.npz'))['arr']
weighted_average_predictions /= all_model_weights

# Поcтавится в True, если были предсказания моделей только по документам
# с токенами по которым были сомнения
#has_doubts_doc = False
# Поcтавится в True, если были предсказания моделей только по документам
# с уже найденными токенами pii
#has_pii_doc = False
# Поcтавится в True, если короткие документы
has_short_doc = False

# ***************************************************************
# Теперь дополнительно получаем предсказания с новыми моделя уже только в документах,
# где ранее нашли какоую то персональную информацию. Потому что таких документов меньше чем всех
# и можно будет прогнать их по большему числу и, возможно, более тяжелых моделей
# ***************************************************************
#! preds_pii = get_preds_final(threshold=find_pii_docs_threshold)
# Получаем список в котором единицами указаны документы с персоанльной информацией
# Получаем финальные прдесказания
#! docs_with_pii = get_docs_with_pii(preds_pii)
#! del preds_pii

#docs_with_pii = get_docs_in_range(0, round(len(doc_list)*range_share))
#docs_with_doubts = get_docs_with_doubts()

# Доля докуметнтов с токенами pii
#! share = sum(docs_with_pii) / len(docs_with_pii)
# estimate_model_time = max_model_time * share

docs_with_short = get_docs_in_range(0, round(len(doc_list)*range_share))
estimate_model_time = max_model_time * range_share
print('estimate_model_time:',estimate_model_time)
cur_time = time.time()
print(p_time())
# Проверяем остались ли еще модели для предсказания и есть ли еще время
if ((last_idx + 1 < len(model_paths))
    and ((cur_time - notebook_starttime) < (short_docs_time_limit - estimate_model_time))):
#if 1 == 2:
    # Если еще остались модели для тренировки
    print(p_time(), 'Тренируем модели только на коротких документах')
    has_short_doc = True
    del weighted_average_predictions
    # Делаем датасет только для коротких документов
    make_dataset(docs_with_short)
    short_model_weights, last_idx, max_model_time = make_sum_predition(
        start_idx=(last_idx + 1),
        docs_to_process=docs_with_short,
        time_limit=short_docs_time_limit,
        sum_predictions_name='short_sum_predictions',
        model_time_mul=1
    )
    del ds
    make_average_predictions()
    
#if has_pii_doc or has_doubts_doc:
if has_short_doc:
    
    #Если переделывали датасет - возвращаем его на место
    #Сохраняем и удаляем веса, чтобы высвободить память перед ресурсоемкой опперацией
    np.savez_compressed(os.path.join(intermediate_dir, f'all_sum_predictions.npz'), arr=weighted_average_predictions)
    del weighted_average_predictions
    # Возвращаем датасет со всеми документами на место
    make_dataset(all_docs)
    weighted_average_predictions = np.load(os.path.join(intermediate_dir, f'all_sum_predictions.npz'))['arr']
# ***************************************************************
# Получаем финальные прдесказания
# ***************************************************************
# Предсказание на основе общего порога для не персональной информации
# preds_final, O_preds = get_preds_final(threshold=final_docs_threshold)

# Предсказание на основе раздельных порогов для каждого типа метки
preds_final, O_preds = get_preds_final_unique()
#docs_with_pii = get_docs_with_pii(preds_final)
del weighted_average_predictions

### Убранные доопредсказяния доп. моделями

In [ ]:
# Пока убрал. Это доработка разных документов, которые не очень сработали.
'''
# ***************************************************************
# Теперь дополнительно получаем предсказания с новыми моделя уже только в документах,
# где ранее нашли какоую то персональную информацию. Потому что таких документов меньше чем всех
# и можно будет прогнать их по большему числу и, возможно, более тяжелых моделей
# ***************************************************************
#! preds_pii = get_preds_final(threshold=find_pii_docs_threshold)
# Получаем список в котором единицами указаны документы с персоанльной информацией
# Получаем финальные прдесказания
#! docs_with_pii = get_docs_with_pii(preds_pii)
#! del preds_pii

docs_with_pii = get_docs_in_range(0, round(len(doc_list)*range_share))
docs_with_doubts = get_docs_with_doubts()
# Доля докуметнтов с токенами pii
#! share = sum(docs_with_pii) / len(docs_with_pii)
#1 estimate_model_time = max_model_time * share
estimate_model_time = max_model_time * range_share
print('estimate_model_time:',estimate_model_time)
cur_time = time.time()
print(p_time())
# Проверяем остались ли еще модели для предсказания и есть ли еще время
if ((last_idx + 1 < len(model_paths))
    and ((cur_time - notebook_starttime) < (pii_docs_time_limit - estimate_model_time))):
#if 1 == 2:
    # Если еще остались модели для тренировки
    print(p_time(), 'Тренируем модели только на документах с pii')
    has_pii_doc = True
    del weighted_average_predictions
    # Делаем датасет только для документов где нашли персональную информацию
    make_dataset(docs_with_pii)
    pii_model_weights, last_idx, max_model_time = make_sum_predition(
        start_idx=(last_idx + 1),
        docs_to_process=docs_with_pii,
        time_limit=pii_docs_time_limit,
        sum_predictions_name='pii_sum_predictions',
        model_time_mul=1
    )
    del ds
    make_average_predictions()

# ***************************************************************
# Теперь дополнительно получаем предсказания с новыми моделя уже только в документах,
# где ранее нашли какоую то персональную информацию. Потому что таких документов меньше чем всех
# и можно будет прогнать их по большему числу и, возможно, более тяжелых моделей
# ***************************************************************
# Доля докуметнтов с сомнениями
share = sum(docs_with_doubts) / len(docs_with_doubts)
estimate_model_time = max_model_time * share
print('estimate_model_time:',estimate_model_time)
cur_time = time.time()
print(p_time())
# Проверяем остались ли еще модели для предсказания и есть ли еще время
if ((last_idx + 1 < len(model_paths))
    and ((cur_time - notebook_starttime) < (doubts_docs_time_limit - estimate_model_time))):
    print(p_time(), 'Тренируем модели только на документах в которых есть токены с сомнениями')
    has_doubts_doc = True

    del weighted_average_predictions
    # Делаем датасет только для документов где нашли персональную информацию
    make_dataset(docs_with_doubts)
    doubts_model_weights, last_idx, max_model_time = make_sum_predition(
        start_idx=(last_idx + 1),
        docs_to_process=docs_with_doubts,
        time_limit=doubts_docs_time_limit,
        sum_predictions_name='doubts_sum_predictions',
        model_time_mul=1
    )
    del ds
    make_average_predictions()
'''
pass

### Перенос предсказаний на исходную токенизацию

In [ ]:
%%time
triplets = []  # Инициализируем список для троек (в данном случае не используется)
pairs = set()  # Используем множество для пар, чтобы проверка наличия элемента была быстрее O(1), чем в списке O(n)

processed = []  # Список для обработанных записей

# Для каждого прогноза, соответствия токенов, смещений, токенов и документа в наборе данных
#for p, token_map, offsets, tokens, doc, doc_with_pii in zip(preds_final, ds["token_map"], ds["offset_mapping"], ds["tokens"], ds["document"], docs_with_pii):
for p, O_p, token_map, offsets, tokens, doc in zip(preds_final, O_preds, ds["token_map"], ds["offset_mapping"], ds["tokens"], ds["document"]):

    #if doc_with_pii == 0:
        # Если в документе нет перс
            #continue
    token_pred_ind = 0 # Индекс токена предсказания

    if is_local:
        index_to_change = df_result.index[df_result['document'] == doc][0]
    # Итерируемся по каждому прогнозу токена и соответствующим смещениям
    for token_pred, O_pred, (start_idx, end_idx) in zip(p, O_p, offsets):
        token_pred_ind += 1
        label_pred = id2label[str(token_pred)]  # Получаем предсказанную метку для токена

        # Если сумма начального и конечного индекса равна нулю, переходим к следующей итерации
        if start_idx + end_idx == 0:
            continue

        # Если соответствие токена при начальном индексе равно -1, увеличиваем начальный индекс
        if token_map[start_idx] == -1:
            start_idx += 1

        # Игнорируем начальные пробельные токены ("\n\n")
        while start_idx < len(token_map) and tokens[token_map[start_idx]].isspace():
            start_idx += 1

        # Если начальный индекс превышает длину соответствия токенов, прерываем цикл
        if start_idx >= len(token_map):
            break

        token_id = token_map[start_idx]  # Получаем ID токена при начальном индексе
                
        """
            {'B-EMAIL',
             'B-ID_NUM',
             'B-NAME_STUDENT',
             'B-PHONE_NUM',
             'B-STREET_ADDRESS',
             'B-URL_PERSONAL',
             'B-USERNAME',
             'I-ID_NUM',
             'I-NAME_STUDENT',
             'I-PHONE_NUM',
             'I-STREET_ADDRESS',
             'I-URL_PERSONAL',
             'O'}
        """

        if is_local:
            # Устанавливаем связь между индексами предсказаний и индексами токенов в тесте на старом токенизаторе
            df_result.loc[index_to_change, 'pred_ind'][token_id] = token_pred_ind - 1
        
        # Игнорируем предсказания "O" и токены пробелов, а также специфичные метки, например "B-EMAIL", "B-PHONE_NUM", "I-PHONE_NUM"
        #if label_pred in ("O", "B-EMAIL", "B-PHONE_NUM", "I-PHONE_NUM") or token_id == -1:
        if label_pred in ("O") or token_id == -1:
            continue

        pair = (doc, token_id)  # Формируем пару из документа и ID токена

        # Если такой пары нет в множестве, добавляем запись в список обработанных
        if pair not in pairs:
            processed.append({"document": doc, "token": token_id, "label": label_pred, "token_str": tokens[token_id], "o_pred": O_pred})
            pairs.add(pair)  # Добавляем пару в множество, чтобы избежать повторений


### Нахождение токенов и телефонов регулярными выражениями

In [ ]:
from spacy.lang.en import English
nlp = English()

def find_span(target: list[str], document: list[str]) -> list[list[int]]:
    idx = 0
    spans = []
    span = []

    for i, token in enumerate(document):
        if token != target[idx]:
            idx = 0
            span = []
            continue
        span.append(i)
        idx += 1
        if idx == len(target):
            spans.append(span)
            span = []
            idx = 0
            continue
    
    return spans

In [ ]:
# Компиляция регулярных выражений для поиска PII
email_regex = re.compile(r'[\w.+-]+@[\w-]+\.[\w.-]+')
phone_num_regex = re.compile(r"(\(\d{3}\)\d{3}\-\d{4}\w*|\d{3}\.\d{3}\.\d{4})\s")

# Инициализация списков для хранения найденных данных
emails = []
phone_nums = []

# Функция для поиска индексов токенов, соответствующих целевой последовательности
def find_span(target, document):
    spans = []
    target_len = len(target)
    for i in range(len(document) - target_len + 1):
        if document[i:i + target_len] == target:
            spans.append(range(i, i + target_len))
    return spans

#!
# Обработка каждого документа в данных
#for _data in data:
for _data in test_data:
    try:
        # Поиск и добавление адресов электронной почты
        for token_idx, token in enumerate(_data["tokens"]):
            if email_regex.fullmatch(token):
                emails.append(
                    {"document": _data["document"], "token": token_idx, "label": "B-EMAIL", "token_str": token, "o_pred": 1}
                )

        # Поиск телефонных номеров в полном тексте документа
        matches = phone_num_regex.findall(_data["full_text"])
        for match in matches:
            target = [t.text for t in nlp.tokenizer(match)]
            matched_spans = find_span(target, _data["tokens"])

            # Добавление информации о телефонных номерах
            for matched_span in matched_spans:
                for intermediate, token_idx in enumerate(matched_span):
                    prefix = "I" if intermediate else "B"
                    phone_nums.append(
                        {"document": _data["document"], "token": token_idx, "label": f"{prefix}-PHONE_NUM", "token_str": _data["tokens"][token_idx], "o_pred": 1}
                    )
    except Exception as e:
        print(f"Ошибка при обработке документа {_data['document']}: {e}")

# Вывод результатов для демонстрации
print(f"Найдено адресов электронной почты: {len(emails)}")
print(f"Найдено телефонных номеров: {len(phone_nums)}")

pass

## Построцессинг
Обрабатываем найденные персональные токены на предыдущих этапах их поиска нейронными сетями и шаблонами

### Добавление токенов с адресами

In [ ]:
%%time
# Пометим пропущенные токены в песональных адресах
# Дополнительные токены для адресов
streets_add = []
prev_label = ''
prev_token = ''
for row in processed:
    if row['label'] == 'I-STREET_ADDRESS' and prev_label == 'I-STREET_ADDRESS':
        # Если предыдущий и последующий токены отмечены как I-STREET_ADDRESS то помечаем токен между ними тем же.
        if row['token'] == prev_token + 2:
            # Если между токенами помечеными I-STREET_ADDRESS есть неразмеченный токен то добавим его
            doc_index = doc_list.index(row['document'])
            # Строка токена которую добавляем
            #token_str_add = ds['tokens'][doc_index][row['token'] - 1]
            token_str_add = test_data[doc_index]['tokens'][row['token'] - 1]
            print("Добавляем токен:",row['document'], row['token'], f'[{token_str_add}]')
            streets_add.append({"document": row['document'],
                                "token": row['token'] - 1,
                                "label": 'I-STREET_ADDRESS',
                                "token_str": token_str_add,
                                "o_pred": 1})
    prev_label = row['label']
    prev_token = row['token']

In [ ]:
#Создаем предварительный датафрейм для анализа
#df_pred = pd.DataFrame(processed + streets_add + phone_nums + emails)
df_pred = pd.DataFrame(processed + streets_add)

### Чистка токенов
Убираем из найденных персональных токенов, которые не соответсвуют некоторой заранее заданной логике.
Скажем слишком короткие, не соответсвуют шаблону, идут в неправильном порядке или что-то подобное

In [ ]:
processed_copy = processed.copy()

In [ ]:
#processed = processed_copy.copy()

In [ ]:
%%time
i = 0
delete_count = 0 # Число удаленных
while i < len(processed):
    is_del_el = False # Нужно ли будет удалять из помеченных найденный токен
    # Номер документа
    doc_idx = processed[i]['document']
    # Число токенов в документе
    tokens_in_doc_count = (df_pred['document'] == doc_idx).sum()
    
    # Чистка начальных токенов за которыми не следуют токены продолжения
    if bool(re.search(r'^B-', processed[i]['label'])):
        # Смотрим есть ли продолжение у персональной ифонмации
        is_next_token = False
        if i+1 < len(processed):
            if bool(re.search(r'^I-', processed[i+1]['label'])):
                is_next_token = True
        if not(is_next_token):
            # Усли есть начальный токен и нет у него токена продолжения,
            # то дальше смотрим насколько он удовлетворяет минимальным условиям

            # B-ID_NUM
            if (processed[i]['label']  == 'B-ID_NUM'):
                if not(bool(re.search(r'\d{2}', processed[i]['token_str']))):
                    #Нет ни двух цифр в токене и следующий токен не I-ID_NUM, тогда улаляем
                    is_del_el = True
                if len(processed[i]['token_str']) <=3:
                    is_del_el = True
            
            # B-STREET_ADDRESS
            if (processed[i]['label']  == 'B-STREET_ADDRESS'):
                # Удаляем адрес короче 10 символов
                if len(processed[i]['token_str']) <=10:
                    is_del_el = True

            # B-NAME_STUDENT
            if (processed[i]['label']  == 'B-NAME_STUDENT'):
                if len(processed[i]['token_str']) <=1:
                    is_del_el = True
            
            # B-PHONE_NUM
            if (processed[i]['label']  == 'B-PHONE_NUM'):
                if len(processed[i]['token_str']) <=4:
                    is_del_el = True
            
            # B-PHONE_NUM
            if (processed[i]['label']  == 'B-EMAIL'):
                if not(bool(re.search(r'[\w.+-]+@[\w-]+\.[\w.-]+', processed[i]['token_str']))):
                    is_del_el = True

    # Чистка имен B-NAME_STUDENT не соответствующих шаблону
    if not(is_del_el) and (processed[i]['label']  == 'B-NAME_STUDENT'):
        if not(bool(re.search(r'^[A-Z][a-z]+$', processed[i]['token_str']))):
            # Если имя не соответствует шалблону (Большая буква потом строчные) то удаляем
            is_del_el = True
        # Проверяем, существует ли в датафрейме хотя бы одна строка без метки 'I-NAME_STUDENT' и с номером документа
        if not any((df_pred['document'] == doc_idx) & (df_pred['label'] == 'I-NAME_STUDENT')):
            #Если нет ни одного токена с I-NAME_STUDENT
            name_count = df_pred[df_pred['document'] == doc_idx]['token_str'].value_counts().get(processed[i]['token_str'], 0)
            # print(doc_idx, processed[i]['token_str'], name_count)
            #if tokens_in_doc_count == name_count and name_count >= 3:
            # Если превышено минимальной число одинаковых имен min_name_count, то удаляем
            min_name_count = 1
            if name_count >= min_name_count:
                min_value = df_pred[(df_pred['document'] == doc_idx) & (df_pred['token_str'] == processed[i]['token_str'])]['o_pred'].min()
                #if processed[i]['o_pred'] > 0.2:
                if min_value > 0.4:
                    # Если вероятность что это было обычный токен достаточно высока
                    #is_del_el = True
                    pass
    
    # Чистка имен I-NAME_STUDENT не соответствующих шаблону
    if not(is_del_el) and (processed[i]['label']  == 'I-NAME_STUDENT'):
        if not(bool(re.search(r'^[A-Z][a-z\.]+$', processed[i]['token_str']))):
            # Если имя не соответствует шалблону (Большая буква потом строчные) то удаляем
            is_del_el = True

        # Чистка I-NAME_STUDENT без началльного токена
        if i > 0:
            if not(((processed[i-1]['label'] == 'I-NAME_STUDENT')
                   or (processed[i-1]['label'] == 'B-NAME_STUDENT'))
                  and ((processed[i]['token'] - processed[i-1]['token']) < 7)):
                # Если продолжение имени идет без начала и чтобы не далеко, не дальше 7 докенов, то удаляем его
                is_del_el = True

    # Чистка слишком коротких B-URL_PERSONAL
    if not(is_del_el) and (processed[i]['label']  == 'B-URL_PERSONAL'):
        if len(processed[i]['token_str']) <=1:
            # Если слишком короткая url - удаляем
            is_del_el = True

        if bool(re.search(r'(youtu\.be|youtube\.com)', processed[i]['token_str'])):
            # Если ютуб
            if tokens_in_doc_count == 1:
                # Если кроме ютуба ничего нет
                if processed[i]['o_pred'] > 0.2:
                    # Если вероятность что это было обычный токен достаточно высока
                    #is_del_el = True
                    pass
            
    if is_del_el:
        print('удаляем:',processed[i]['label'],processed[i]['token_str'],'doc:', doc_idx, 'o_pred:', processed[i]['o_pred'])
        del processed[i]
        delete_count += 1
    else:
        i += 1
print('Всего удалили', delete_count)

### Удаляем слишком длинные списки имен

In [ ]:
# Удяляем слишком длинные списки имен которые модель приняла за персональные данные,
# Но это может быть список литературы или что-то подобное

def del_names_list(processed):
    tokens_to_del = [] # список индексов токенов в списке processed, которые нужно удалить
    candidates = [] # список кандидатов на добавление в список для удаления
    candidates_names = [] # список кандидатов на добавление в список для удаления
    b_name_count = -1 # Число именно начальных токенов в последовательности имен
    last_doc = -1 # Последний обработанный документ
    last_token = -1 # Последний обработанный токен
    for i in range(len(processed)):
        if ((processed[i]['label'] == 'I-NAME_STUDENT')
            or (processed[i]['label'] == 'B-NAME_STUDENT')):
               
            # Если список имен уже начался проверяем насколько далеко 
            # отстоит этот токен от предыдущего токена с именем.
            # Нужно чтобы отстоял не более чем на два токена
            token_diff = 5 # Допустимое число токенов между именами, чтобы это была еще последовательность
            if  ((processed[i]['document'] == last_doc)
                 and ((last_token + token_diff) >= processed[i]['token'])):
                if (processed[i]['label'] == 'B-NAME_STUDENT'):
                    b_name_count += 1
            else:
                # Список имен прекратился
                if b_name_count > 2:
                    # Нашли именно несколько подряд токенов с именами, добавляем их в список для удаления
                    is_duplicates = False
                    # Но сначала посмотрим нет ли повторяющихся имен в списке
                    for name in candidates_names:
                        if candidates_names.count(name) > 1:
                            is_duplicates = True
                    if not (is_duplicates):
                        tokens_to_del.extend(candidates)
                candidates = []
                candidates_names = []
                if (processed[i]['label'] == 'B-NAME_STUDENT'):
                    b_name_count = 1

            
            #Действия необходимые произвести при начале или продолжении списка имен
            last_doc = processed[i]['document']
            last_token = processed[i]['token']
            candidates.append(i)
            candidates_names.append(processed[i]['token_str'])
    return(tokens_to_del)

In [ ]:
'''
# Создаем список для удаления лишних токенов с именами, которых много подряд
tokens_to_del = del_names_list(processed)
# Удаляем токены с именами подряд из помеченных как персональная информация
processed = [el for i, el in enumerate(processed) if i not in tokens_to_del]
print('Удалено помеченнных как pii токенов с именами идущих подряд:', len(tokens_to_del))
'''
pass

In [ ]:
# processed = processed_copy.copy()

### Сохранение сабмита

In [ ]:
#df = pd.DataFrame(processed + streets_add + phone_nums + emails)
df = pd.DataFrame(processed + streets_add)
#df = pd.DataFrame(processed + streets_add)

# Assign each row a unique 'row_id'
df["row_id"] = list(range(len(df)))

# Display a glimpse of the first 100 rows of your data
display(df.head(100))

# Cast your findings into a CSV file for further exploration
df[["row_id", "document", "token", "label"]].to_csv("submission.csv", index=False)

## Вывод результатов
Выводим те токены с персональной информацнией, что не удалось найти или что нашли лишнее с подробным описанием.

In [ ]:
# Выводет токены вокруг указанного чтобы посмотреть контекст
def print_fragment():
    if is_print_details:
        print('Фрагмент:')
    fragment_len = 100 # Устанавливаем что будем выводить токены 7 до и 7 после
    token_arr = []
    label_arr = []
    for i in range(max(0, token_id - fragment_len),
                   min(len(tokens) - 1, token_id + fragment_len)):
    #for i, token in enumerate(tokens[token_id - fragment_len:token_id + fragment_len]):
        token = tokens[i]
        token = token.replace('\n', '\\n')
        if i == token_id:
            # Помечаем зеленым цветом если нашли лишеюю персональную информацию
            # Помечаем красным цветом если если не персональную информацию
            color = ''
            if true_label == "O":
                # Если истинная метка токена была что это не персональная ифномация
                color = Fore.GREEN
                metrics['FP'] += 1
            elif pred_label == "O":
                # Если истинная метка токена была что это персональная ифномация,
                # но модель определила как не персональныю информацию
                color = Fore.RED
                metrics['FN'] += 1
            else:
                # И истинная метка было что персональня инфа и модель так определила что персональня инфа
                # Но модель не правильно определила тип персональной инфф для токена
                metrics['err_dist'] += 1
                
            color = Fore.GREEN if true_label == "O" else (Fore.RED if pred_label == "O" else '')
            excl = '' if true_label == "O" else ('!' if pred_label == "O" else '?')
                
            token_str = f'***{color}{excl}{token}{Style.RESET_ALL}***'
        else:
            token_str = f"{token}"
        token_arr.append(f'{token_str} ')
        label_arr.append(f'[{token_str}]{true_labels[i]}, ')

    if is_print_details:
        # Выводим просто строку в контексте
        out = ''
        for el in token_arr:
            out += el
        print(out)
        print()
        # Выводим просто токены строки с тру метками
        out = ''
        for el in label_arr:
            out += el
        print(out)

In [ ]:
if is_local:
#if 1==2:
    # Добавим к df_result метки к токенам с персональной ифонманицией, которые нашла модель
    for row in df.itertuples():
        
        # Находим индекс документа в датасете
        index_to_change = df_result.index[df_result['document'] == row.document][0]
        df_result.loc[index_to_change, 'pred_labels'][row.token] = row.label
        # df_result.loc[index_to_change, 'pred_token_str'][row.token] = row.token_str
        # print(doc_index, row.token, row.document, row.label)
        #break
    labels = [
        'B-EMAIL',
        'B-ID_NUM',
        'B-NAME_STUDENT',
        'B-PHONE_NUM',
        'B-STREET_ADDRESS',
        'B-URL_PERSONAL',
        'B-USERNAME',
        'I-ID_NUM',
        'I-NAME_STUDENT',
        'I-PHONE_NUM',
        'I-STREET_ADDRESS',
        'I-URL_PERSONAL',
        'O'
    ]
    
    processed_result = []  # Список для обработанных записей
    
    metrics = {
        'FP': 0, # Ошибочно пометил токен что персональная ифнормация
        'TP': 0, # Проавильно пометил токен с персональной информацией
        'FN': 0, # Не пометил токен с персональной информацией
        'TN': 0, # Проавильно пометил токен с не персональной информацией
        'err_dist': 0, # Нашел что токен персональная ифонмация но не того типа
        'err_all': 0, # Всего ошибок
        'torekn_all': 0 # Всего токенов
    }

    # Число ошибок в разрезе отдельных меток
    label_err_count = {key: 0 for key in labels}
    # Инициализируем число истинных токенов в разрезе по меткам
    true_label_count = {key: 0 for key in labels}
    # Инициализируем минимальную длину истинных токенов в разрезе по меткам
    true_label_min_len = {key: 100 for key in labels}
    name_str = []

    
    # Пройдемся по предсказаниям и посмотрим где они не совпадают с истинными метками токенов.
    # Там где на совпадает 
    if is_print_details:
        print('Не совпадение предсказаний с истинными метками токенов:\n')
    
    doc_idx = 0 # Индекс документа
    for tokens, doc, true_labels, pred_labels, pred_indes in zip(
        df_result["tokens"],
        df_result["document"],
        df_result["true_labels"],
        df_result["pred_labels"],
        df_result["pred_ind"]):
    
        # Номер токена
        token_id = 0

        #Сколько токенов в документе
        true_labels_len = len(true_labels)

        # Итерируемся по каждому прогнозу токена и соответствующим смещениям
        for token, true_label, pred_label in zip(tokens, true_labels, pred_labels):
            metrics['torekn_all'] += 1
            true_label_count[true_label] += 1
            # Находим минимальную длину истинного токена
            # При условии что дальше нет продолжения
            is_next_token = False
            if token_id + 1 < len(tokens):
                if bool(re.search(r'^I-', true_labels[token_id + 1])):
                    is_next_token = True
            if not(is_next_token):
                true_label_min_len[true_label] = len(token) if len(token) < true_label_min_len[true_label] else true_label_min_len[true_label]
                if true_label == 'B-NAME_STUDENT':
                    if len(token) <=2:
                        #name_str.append([doc,token_id,len(token),token])
                        pass
            if true_label == 'B-NAME_STUDENT':
                name_str.append([doc,token_id,len(token),token])
            # if (true_label not in ("O")) or (pred_label not in ("O")):
            if not(true_label == pred_label):
                # если определеили токен не правильно
                
                metrics['err_all'] += 1
                label_err_count[pred_label] += 1
                
                if is_print_details: 
                    print(f'\n******************************************** {metrics["err_all"]} ********************************************\n')
                    print(pd.DataFrame({"document": [doc],
                                        "doc_len": [true_labels_len],
                                        "token": [token_id],
                                        #"true_label": [true_label],
                                        "true_label-pred_label-len": [f'{true_label} {pred_label} {len(token)}'],
                                        #"token_len": [len(token)],
                                        "token_str": [token]})
                          .to_string(index=False))
                # Печатаем фрагмент текста c токеном по середине который не правильно определили
                print_fragment()

                # pred_ind -индекс текущего токена в токенах предсказания (токены модели и тестовые токены могут отличаться в том числе по номеру в тексте)
                pred_ind = pred_indes[token_id] 

                if pred_ind > 0:
                    
                    # Создаем датафрейм где будут предсказания моделей по меткам
                    pred_mon_df = pd.DataFrame({'Labels': labels})
                    #if is_print_details:
                    if 1 == 2:
                        # Выведем предсказания всех моделей для всех возможных классов для текущего токена
                        for model_idx in range(len(model_predictions)):
                            pred_mon_df[f'model_{model_idx}'] = [round(num * 100) 
                                                                 for num in model_predictions[model_idx][doc_idx][pred_ind]]
                        pred_mon_df[f'av_pred'] = [round(num * 100)
                                                   for num in weighted_average_predictions[doc_idx][pred_ind]]
                        display(pred_mon_df)
                else:
                    if is_print_details:
                        print('Не нашел индекс предсказания для токена!!!')
            else:
                # если определеили токен правильно
                if pred_label == 'O':
                    metrics['TN'] += 1
                else:
                    metrics['TP'] += 1
            token_id += 1
        doc_idx += 1
    # Считаем точность
    try:
        metrics['precision'] = metrics['TP']/(metrics['TP'] + metrics['FP'])
    except:
        metrics['precision'] = 0.0
    
    # Считаем полноту
    try:
        metrics['recall'] = metrics['TP']/(metrics['TP'] + metrics['FN'])
    except:
        metrics['recall'] = 0.0
    
    # Считаем F5
    try:
        metrics['F5'] = (26 * metrics['precision'] * metrics['recall']) / (25 * metrics['precision'] + metrics['recall'])
    except:
        metrics['F5'] = 0.0

    print('\n------------------------------------------------------------------------------------\n')
    print('Минимальные длины токенов:')
    display(true_label_min_len)
    
    print('\n------------------------------------------------------------------------------------\n')
    print('Всего истинных токенов в разрезе по меткам:')
    display(true_label_count)
    
    print('\n------------------------------------------------------------------------------------\n')
    print('Ошибкок предсказний модели в разрезе по меткам:')
    display(label_err_count)
    
    print('\n------------------------------------------------------------------------------------\n')
    print ('Всего токенов:', metrics['torekn_all'])
    print ('Не отметил токены с персональной информацией (FN):', metrics['FN'])
    print ('Пометил лишних токенов (FP):', metrics['FP'])
    print ('Не правильно определил тип персональной информации:', metrics['err_dist'])
    print ('precision:', metrics['precision'])
    print ('recall:', metrics['recall'])
    print ('F5:', metrics['F5'])

In [ ]:
if is_local:
    cols = ['doc', 'token', 'len', 'token_str']
    name_str_df = pd.DataFrame(name_str, columns=cols)
    name_str_df_grouped = name_str_df.groupby('doc')['token_str'].count()
    display(name_str_df_grouped.value_counts())

In [ ]:
if is_local:
    for tokens, doc, true_labels, pred_labels, pred_indes in zip(
        df_result["tokens"],
        df_result["document"],
        df_result["true_labels"],
        df_result["pred_labels"],
        df_result["pred_ind"]):
        true_labels_len = len(true_labels)
        for idx, (token, true_label, pred_label) in enumerate(zip(tokens, true_labels, pred_labels)):
            if true_label != 'O':
                print ('doc:',doc, 'true_label:',true_label, 'token idx:', idx, 'tokens all:', true_labels_len)
        

In [ ]:
print(p_time(), 'end notebook')

In [ ]:
#df

In [ ]:
if is_local:
    for tokens, doc, true_labels, pred_labels, pred_indes in zip(
        df_result["tokens"],
        df_result["document"],
        df_result["true_labels"],
        df_result["pred_labels"],
        df_result["pred_ind"]):
        true_labels_len = len(true_labels)
        for idx, (token, true_label, pred_label) in enumerate(zip(tokens, true_labels, pred_labels)):
            if true_label != 'O':
                print ('doc:',doc, 'true_label:',true_label, 'token idx:', idx, 'tokens all:', true_labels_len)
                

In [ ]:
if is_local:
    for tokens, doc, true_labels, pred_labels, pred_indes in zip(
        df_result["tokens"],
        df_result["document"],
        df_result["true_labels"],
        df_result["pred_labels"],
        df_result["pred_ind"]):
        true_labels_len = len(true_labels)
        for idx, (token, true_label, pred_label) in enumerate(zip(tokens, true_labels, pred_labels)):
            if true_label != 'O' or pred_label != 'O':
                print ('doc:',doc, 'token:', token, ', true_label:',true_label, 'pred_label:',pred_label, 'token idx:', idx, 'tokens all:', true_labels_len)

In [ ]:
if is_local:
    print(test_data[doc_list.index(9405)]['full_text'])